In [69]:
####################################################################
# If you have any questions, please, contant me via anna@aconit.nl #
# Anna                                                             #
####################################################################

%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [72]:
import math

# Network Node class :
class QNetworkNode:
 def __init__(self, name, index):
    self.name = name
    self.index = index


 def sendMessage(self, network):
    self.coincircuit.h(0)
    
    #Prepare the Coin circuit
    qubitsamount = self.coincircuit.n_qubits
    
    for num in range(1,qubitsamount):
        self.coincircuit.cx(0,num)
    print("Coincircuit")
    print(self.coincircuit)
    
    #Prepare the Leader circuit
    for num in range(0,int(network.leaderqubitamount)):
        self.leadercircuit.h(int(num))
        for j in range(1, network.nodesnumber):
            self.leadercircuit.cx(int(num),int( j*network.leaderqubitamount))
    print("Leadercircuit")
    print(self.leadercircuit)

 def receiveMessage(self, network):
    coinvalues=[]
    leadervalues =[]
    for node in network.nodes:
        currResult = str(node.coinresult)
        coinvalues.append(currResult[self.index+1:self.index+2])
        leaderResult = str(node.leaderresult)
        print(leaderResult)
        leaderResult=leaderResult.replace("{", "").replace(",", "").replace("'", "")
        currIndex = int(self.index-1)
        leadervalues.append(leaderResult[int(currIndex *network.leaderqubitamount):int(currIndex *network.leaderqubitamount +network.leaderqubitamount) ])    
    print ("coinvalues")
    print (coinvalues)
    print("leadervalues")
    print(leadervalues)
    
    #find the index of the leader
    i=0
    maxsum=0
    desiredindex = 0
    for num in leadervalues:
        i+=1
        sumvalue=0
        for s in str(num):
            if s=="1":
                sumvalue+=1
        if sumvalue>maxsum:
            maxsum=sumvalue
            desiredindex = i
    
    print("maxsum: " + str(maxsum))
    print("desiredindex: " + str(desiredindex))
    print("TA-DA!!!! Leaders coin: " + str(coinvalues[desiredindex]))
            

        

# Network class
class QNetwork:  
 def __init__(self, nodesnumber):
    self.nodesnumber = nodesnumber
    self.nodes = []
    
    leadervalue = nodesnumber**3
    leaderqubitamount = math.log(leadervalue,2)
    self.leaderqubitamount = leaderqubitamount
    leaderqubitamount*=nodesnumber
    #print(leaderqubitamount)
        
    for num in range(0,nodesnumber):
        ordernumber = num+1
        newNode = QNetworkNode("node"+ str(ordernumber), ordernumber)
        newNode.coincircuit = QuantumCircuit(nodesnumber,nodesnumber)
        newNode.leadercircuit = QuantumCircuit(int(leaderqubitamount),int(leaderqubitamount))
        self.nodes.append(newNode)

In [73]:
MyNetwork = QNetwork(4)

#send the messages (Coins and Leader states)
for node in MyNetwork.nodes: 
    node.sendMessage(MyNetwork)
 
 #do the measurement
for node in MyNetwork.nodes: 
 node.coincircuit.measure(range(MyNetwork.nodesnumber),range(MyNetwork.nodesnumber))
 simulator = Aer.get_backend('qasm_simulator')
 coinjob = execute(node.coincircuit, simulator, shots=1)
 coinresult = coinjob.result()
 node.coinresult = coinresult.get_counts(node.coincircuit)

 node.leadercircuit.measure(range(node.leadercircuit.n_qubits),range(node.leadercircuit.n_qubits))
 leaderjob = execute(node.leadercircuit, simulator, shots=1)
 leaderresult = leaderjob.result()
 node.leaderresult = leaderresult.get_counts(node.leadercircuit)
 
#read the results
for node in MyNetwork.nodes:     
 node.receiveMessage(MyNetwork)


Coincircuit
        ┌───┐               
q_0: |0>┤ H ├──■────■────■──
        └───┘┌─┴─┐  │    │  
q_1: |0>─────┤ X ├──┼────┼──
             └───┘┌─┴─┐  │  
q_2: |0>──────────┤ X ├──┼──
                  └───┘┌─┴─┐
q_3: |0>───────────────┤ X ├
                       └───┘
 c_0: 0 ════════════════════
                            
 c_1: 0 ════════════════════
                            
 c_2: 0 ════════════════════
                            
 c_3: 0 ════════════════════
                            
Leadercircuit
         ┌───┐                                                                 »
 q_0: |0>┤ H ├──■────■─────────■───────────────────────────────────────────────»
         ├───┤  │    │         │                                               »
 q_1: |0>┤ H ├──┼────┼────■────┼────■─────────■────────────────────────────────»
         ├───┤  │    │    │    │    │         │                                »
 q_2: |0>┤ H ├──┼────┼────┼────┼────┼────■────┼────■─────────■──────────────